In [1]:
import pandas as pd
import json
import numpy as np
import jenks

In [2]:
momc = pd.read_csv('../scratch/momc_meta.csv').set_index('KOD_MOaMC').to_dict(orient='index')
ob = pd.read_csv('../scratch/obce_meta.csv').set_index('KOD_OBEC').to_dict(orient='index')

In [3]:
with open('../scratch/okrsky.json', encoding='utf-8') as f:
    jsn = json.load(f)

In [4]:
meta = {}

In [5]:
# zpracovani MC
for mc in momc:
    meta[mc] = {
        'mc': momc[mc]['NAZ_MOaMC'], 
        'obec': momc[mc]['NAZ_OBEC'], 
        'okres': momc[mc]['NAZ_LAU1'], 
        'kraj': momc[mc]['NAZ_CZNUTS3']
    }

In [6]:
# zpracovani obci
for o in ob:
    meta[o] = {
        'mc': None, 
        'obec': ob[o]['NAZ_OBEC'], 
        'okres': ob[o]['NAZ_LAU1'], 
        'kraj': ob[o]['NAZ_CZNUTS3']
    }

In [7]:
for fe in jsn['features']:
    momc_kod = fe['properties']['MomcKod']
    obec_kod = fe['properties']['ObecKod']
    if momc_kod != None:
        kd = momc_kod
    else:
        kd = obec_kod
    
    fe['properties'].update(meta[kd])

In [8]:
okrs = pd.read_csv('../scratch/okrsky_temp.csv')

In [9]:
okrs

,OBEC,OKRSEK,DATUM_CAS_ZPRAC,HLASY_07,HLASY_09,ODEVZ_OBAL,OPAKOVANE,PL_HL_CELK,PORADI_ZPRAC,VYD_OBALKY,VOL_SEZNAM
0,541281,5,2018-01-27T14:16:05,38,16,54,0,54,2,54,123
1,549584,2,2018-01-27T14:16:15,24,34,58,0,58,3,58,79
2,547701,1,2018-01-27T14:16:19,19,28,47,0,47,4,47,68
3,587206,1,2018-01-27T14:16:31,24,23,47,0,47,5,47,66
4,573060,5,2018-01-27T14:16:33,46,31,77,0,77,6,77,123
5,575500,8,2018-01-27T14:16:37,29,11,40,0,40,7,40,64
6,563960,3,2018-01-27T14:16:37,23,22,45,0,45,8,45,82
7,597678,6,2018-01-27T14:16:38,25,5,30,0,30,9,30,35
8,572667,1,2018-01-27T14:16:48,74,51,125,0,125,10,125,171
9,572241,11,2018-01-27T14:17:00,32,3,35,0,35,11,35,44


In [10]:
okrs['okid'] = okrs.OBEC.astype('str') + '_' + okrs.OKRSEK.astype('str')

In [11]:
okrs.columns

Index(['OBEC', 'OKRSEK', 'DATUM_CAS_ZPRAC', 'HLASY_07', 'HLASY_09',
       'ODEVZ_OBAL', 'OPAKOVANE', 'PL_HL_CELK', 'PORADI_ZPRAC', 'VYD_OBALKY',
       'VOL_SEZNAM', 'okid'],
      dtype='object')

In [12]:
cls = [
    'CHYBA',
    'KC_1',
    'KC_2',
    'KC_3',
    'KC_4',
    'HLASY_10',
    'HLASY_11',
    'HLASY_12',
    'HLASY_13',
    'HLASY_14',
    'HLASY_15',
    'HLASY_16',
    'HLASY_17',
    'HLASY_18',
    'HLASY_19',
    'HLASY_20',
    'KC_SUM',
    'KOLO',
    'OBEC',
    'OKRES',
    'OKRSEK',
    'OPRAVA',
    'POSL_KAND',
    'TYP_FORM'
]
okrs.drop(cls, inplace=True, axis=1)

ValueError: labels ['CHYBA' 'KC_1' 'KC_2' 'KC_3' 'KC_4' 'HLASY_10' 'HLASY_11' 'HLASY_12'
 'HLASY_13' 'HLASY_14' 'HLASY_15' 'HLASY_16' 'HLASY_17' 'HLASY_18'
 'HLASY_19' 'HLASY_20' 'KC_SUM' 'KOLO' 'OKRES' 'OPRAVA' 'POSL_KAND'
 'TYP_FORM'] not contained in axis

In [13]:
okrs = okrs.set_index('okid').to_dict(orient='index')

In [14]:
for fe in jsn['features']:
    momc_kod = fe['properties']['MomcKod']
    obec_kod = fe['properties']['ObecKod']
    if momc_kod != None:
        kd = str(momc_kod) + '_' + str(fe['properties']['Cislo'])
    else:
        kd = str(obec_kod) + '_' + str(fe['properties']['Cislo'])
    
    if kd in okrs:
        fe['properties'].update(okrs[kd])
    else:
        print(kd)

500097_3045
592935_1
545422_1
555177_1
500119_4109
547034_11015
582824_1
582824_2
503941_1


In [15]:
with open('../scratch/map.json', 'w', encoding='utf-8') as f:
    f.write(json.dumps(jsn))

In [16]:
#breaks
k = 5
nbAttempt = 4

target = open('../js/breaks.js', 'w')
target.truncate()
target.write('var breaks = {\n')

for i in range(1, 10):
    print('kandidat ' + str(i))
    vals = []
    for ftr in jsn['features']:
        try:
            vals.append(ftr['properties']['HLASY_' + str(i).rjust(2, '0')] / ftr['properties']['PL_HL_CELK'])
        except:
            print('omit')
        
    vals = np.trim_zeros(vals)
    if np.mean(vals) == 0:
        continue
    else:
        brks = [0, 0, 0, 0]
        try:
            casp = jenks.jenksCaspall(vals, k, nbAttempt, initStyle='kpp')
            brks = casp.breaks
        except:
            pass
        target.write('"HLASY_' + str(i).rjust(2, '0') + '": ' + str(brks) + ',\n')
        
vals = []
print('ucast')
for ftr in jsn['features']:
    try:
        vals.append(ftr['properties']['PL_HL_CELK'] / ftr['properties']['VOL_SEZNAM'])
    except:
        print('omit')

vals = np.trim_zeros(vals)
if np.mean(vals) == 0:
    pass
else:
    brks = [0, 0, 0, 0]
    try:
        casp = jenks.jenksCaspall(vals, k, nbAttempt, initStyle='kpp')
        brks = casp.breaks
    except:
        pass
    target.write('"UCAST": ' + str(brks) + ',\n')       
target.write("};")
target.close()

kandidat 1
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit

omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit


omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit


omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit


omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit


...

omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit


omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit


omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit


omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit


omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
omit
